In [241]:
import pandas as pd
import numpy as np
trainingSetDF = pd.read_excel("DrugInformation_ImmuneModPaper.xlsx")
###Drop duplicates (certain drugs listed multiple times with slightly different smiles)
trainingSetDF = trainingSetDF.drop_duplicates(subset = "CHEMBL_ID")
##Remove compounds without ChEMBL IDs
trainingSetDF = trainingSetDF[~pd.isna(trainingSetDF.CHEMBL_ID)]
drugBankDF = pd.read_csv("structure links_1.csv")
##Remove compounds without ChEMBL IDs
drugBankDF = drugBankDF[~pd.isna(drugBankDF.CHEMBL_ID)]
##How many drugs overlap between the training set and drugbank?
overlap = len(set(trainingSetDF.CHEMBL_ID.unique()).intersection(set(drugBankDF.CHEMBL_ID.unique())))
print(overlap)
##Merge
combinedDF = pd.merge(trainingSetDF, drugBankDF, on= "CHEMBL_ID", how = "inner")
assert combinedDF.shape[0] == overlap

747


In [242]:
combinedDF.head()

,SMILES_x,CHEMSPIDER_ID,CHEMBL_ID,CHEMBL_KNOWN_DRUG,CHEMBL_NAME,CHEMBL_ALTERNATE_NAME,CLINICALTRIALS_DB,CONCENTRATION_uM,DrugBank ID,Name,...,SMILES_y,Formula,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,HET ID,ChemSpider ID,BindingDB ID
0,BrC1=CN([C@@H]2O[C@H](CO)[C@@H](O)C2)C(=O)NC1=O,5813.0,CHEMBL222280,No,BROXURIDINE,SID50105447|SID17389223|SID90341500|SID5646351...,1,10,DB12028,Broxuridine,...,[H][C@]1(O)C[C@@]([H])(O[C@]1([H])CO)N1C=C(Br)...,C9H11BrN2O5,NaN,NaN,6035.0,347828343.0,472552.0,U33,5813.0,50207303.0
1,Brc1c2nccnc2ccc1NC=1NCCN=1,2341.0,CHEMBL844,Yes,BRIMONIDINE,Alphagan|Brimonidine|UK-14304|Alphagan-P|SID10...,256,10,DB00484,Brimonidine,...,BrC1=C(NC2=NCCN2)C=CC2=NC=CN=C12,C11H10BrN5,C07886,D07540,2435.0,46507305.0,3175.0,NaN,2341.0,34572.0
2,Brc1cc(cc(Br)c1O)C(=O)c1c2c(oc1CC)cccc2,2243.0,CHEMBL388590,No,BENZBROMARONE,SID11110651|SID26664708|SID24424558|SID5010399...,2,10,DB12319,Benzbromarone,...,CCC1=C(C(=O)C2=CC(Br)=C(O)C(Br)=C2)C2=CC=CC=C2O1,C17H12Br2O3,NaN,D01056,2333.0,347828582.0,3023.0,NaN,2243.0,50158460.0
3,Brc1ccc(cc1)[C@H](CCN(C)C)c1ncccc1,16068.0,CHEMBL1201287,Yes,DEXBROMPHENIRAMINE,SID90341837|Dexbrompheniramine|SID29216402|,0,10,DB00405,Dexbrompheniramine,...,CN(C)CC[C@@H](C1=CC=C(Br)C=C1)C1=CC=CC=N1,C16H19BrN2,NaN,NaN,16960.0,46505186.0,59269.0,NaN,16068.0,NaN
4,Brc1ccccc1C[N+](CC)(C)C,2337.0,CHEMBL1199080,Yes,BRETYLIUM,SID50113155|Bretylium|SID50113154|,0,17,DB01158,Bretylium,...,CC[N+](C)(C)CC1=CC=CC=C1Br,C11H17BrN,C06855,D00645,2431.0,46505320.0,3172.0,NaN,2337.0,NaN


In [243]:
## molecular calculator
from rdkit import Chem
from mordred import Calculator, descriptors
calc = Calculator(descriptors, ignore_3D = True)

In [244]:
mols = [Chem.MolFromSmiles(smi) for smi in combinedDF['SMILES_x']]

In [245]:
calDF = calc.pandas(mols)

 28%|██▊       | 207/747 [00:27<02:01,  4.44it/s]

/Users/eric/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 30%|██▉       | 224/747 [00:30<02:45,  3.17it/s]

/Users/eric/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 747/747 [01:46<00:00,  7.03it/s]


In [246]:
calDF

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,13.003721,11.805899,0,0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,...,9.743730,63.794961,305.985134,10.928040,505,26,88.0,104.0,7.027778,3.777778
1,13.423098,11.209659,0,3,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,...,9.736193,63.007560,291.011957,10.778221,515,24,90.0,106.0,4.305556,3.750000
2,17.125041,14.795147,0,0,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,...,10.136938,70.803080,421.915318,12.409274,988,38,118.0,143.0,8.000000,4.861111
3,14.389425,12.216867,0,1,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,...,9.572271,51.676805,318.073161,8.370346,718,25,92.0,103.0,6.305556,4.305556
4,9.579175,9.100648,0,1,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,...,9.256938,43.893749,242.053888,8.068463,248,16,62.0,68.0,5.784722,2.944444
5,23.598392,18.449570,0,1,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,...,10.344545,79.445461,449.118544,8.636895,2633,49,160.0,190.0,9.722222,6.527778
6,30.079292,22.385369,0,0,47.666770,2.391688,4.783375,47.666770,1.191669,4.572573,...,10.406473,76.538683,536.438202,5.587898,7984,57,194.0,216.0,16.013889,8.888889
7,26.472550,20.542158,0,1,44.176297,2.515749,5.031498,44.176297,1.299303,4.454465,...,10.600004,70.685296,476.223056,7.107807,3458,60,180.0,215.0,10.902778,7.486111
8,21.867940,17.016372,0,1,38.234665,2.413270,4.826539,38.234665,1.318437,4.272841,...,10.059765,63.939923,405.185942,7.108525,2488,43,142.0,163.0,8.527778,6.777778
9,3.932653,4.244375,1,0,6.000000,2.000000,4.000000,6.000000,1.000000,2.610845,...,7.626083,30.698690,127.943185,15.992898,29,4,22.0,21.0,4.222222,1.444444


In [247]:
## Adding CHEMBL_ID as index
calDF = calDF.set_index(x for x in combinedDF['CHEMBL_ID'])

In [286]:
calDF.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
CHEMBL222280,13.003721,11.805899,0,0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,...,9.743730,63.794961,305.985134,10.928040,505,26,88.0,104.0,7.027778,3.777778
CHEMBL844,13.423098,11.209659,0,3,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,...,9.736193,63.007560,291.011957,10.778221,515,24,90.0,106.0,4.305556,3.750000
CHEMBL388590,17.125041,14.795147,0,0,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,...,10.136938,70.803080,421.915318,12.409274,988,38,118.0,143.0,8.000000,4.861111
CHEMBL1201287,14.389425,12.216867,0,1,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,...,9.572271,51.676805,318.073161,8.370346,718,25,92.0,103.0,6.305556,4.305556
CHEMBL1199080,9.579175,9.100648,0,1,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,...,9.256938,43.893749,242.053888,8.068463,248,16,62.0,68.0,5.784722,2.944444


In [249]:
calDF.info()

<class 'mordred._base.pandas_module.MordredDataFrame'>
Index: 747 entries, CHEMBL222280 to CHEMBL499
Columns: 1613 entries, ABC to mZagreb2
dtypes: bool(2), float64(894), int64(314), object(403)
memory usage: 9.2+ MB


In [250]:
calDF.columns.to_series().groupby(calDF.dtypes).groups

{dtype('bool'): Index(['Lipinski', 'GhoseFilter'], dtype='object'),
 dtype('int64'): Index(['nAcid', 'nBase', 'nAromAtom', 'nAromBond', 'nAtom', 'nHeavyAtom',
        'nSpiro', 'nBridgehead', 'nHetero', 'nH',
        ...
        'n9FAHRing', 'n10FAHRing', 'n11FAHRing', 'n12FAHRing', 'nG12FAHRing',
        'nRot', 'Diameter', 'Radius', 'WPath', 'WPol'],
       dtype='object', length=314),
 dtype('float64'): Index(['ABC', 'ABCGG', 'SpAbs_A', 'SpMax_A', 'SpDiam_A', 'SpAD_A', 'SpMAD_A',
        'LogEE_A', 'VE1_A', 'VE2_A',
        ...
        'SRW08', 'SRW09', 'SRW10', 'TSRW10', 'MW', 'AMW', 'Zagreb1', 'Zagreb2',
        'mZagreb1', 'mZagreb2'],
       dtype='object', length=894),
 dtype('O'): Index(['AATS5dv', 'AATS6dv', 'AATS7dv', 'AATS8dv', 'AATS5d', 'AATS6d',
        'AATS7d', 'AATS8d', 'AATS5s', 'AATS6s',
        ...
        'MDEO-11', 'MDEO-12', 'MDEO-22', 'MDEN-11', 'MDEN-12', 'MDEN-13',
        'MDEN-22', 'MDEN-23', 'MDEN-33', 'Vabc'],
       dtype='object', length=403)}

In [287]:
## Change data types to float
calDFFloat = pd.DataFrame(calDF, dtype='float')

In [288]:
calDFFloat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 747 entries, CHEMBL222280 to CHEMBL499
Columns: 1613 entries, ABC to mZagreb2
dtypes: float64(1613)
memory usage: 9.2+ MB


In [292]:
calDFFloat

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
CHEMBL222280,13.003721,11.805899,0.0,0.0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,...,9.743730,63.794961,305.985134,10.928040,505.0,26.0,88.0,104.0,7.027778,3.777778
CHEMBL844,13.423098,11.209659,0.0,3.0,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,...,9.736193,63.007560,291.011957,10.778221,515.0,24.0,90.0,106.0,4.305556,3.750000
CHEMBL388590,17.125041,14.795147,0.0,0.0,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,...,10.136938,70.803080,421.915318,12.409274,988.0,38.0,118.0,143.0,8.000000,4.861111
CHEMBL1201287,14.389425,12.216867,0.0,1.0,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,...,9.572271,51.676805,318.073161,8.370346,718.0,25.0,92.0,103.0,6.305556,4.305556
CHEMBL1199080,9.579175,9.100648,0.0,1.0,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,...,9.256938,43.893749,242.053888,8.068463,248.0,16.0,62.0,68.0,5.784722,2.944444
CHEMBL601719,23.598392,18.449570,0.0,1.0,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,...,10.344545,79.445461,449.118544,8.636895,2633.0,49.0,160.0,190.0,9.722222,6.527778
CHEMBL1293,30.079292,22.385369,0.0,0.0,47.666770,2.391688,4.783375,47.666770,1.191669,4.572573,...,10.406473,76.538683,536.438202,5.587898,7984.0,57.0,194.0,216.0,16.013889,8.888889
CHEMBL841,26.472550,20.542158,0.0,1.0,44.176297,2.515749,5.031498,44.176297,1.299303,4.454465,...,10.600004,70.685296,476.223056,7.107807,3458.0,60.0,180.0,215.0,10.902778,7.486111
CHEMBL954,21.867940,17.016372,0.0,1.0,38.234665,2.413270,4.826539,38.234665,1.318437,4.272841,...,10.059765,63.939923,405.185942,7.108525,2488.0,43.0,142.0,163.0,8.527778,6.777778
CHEMBL13960,3.932653,4.244375,1.0,0.0,6.000000,2.000000,4.000000,6.000000,1.000000,2.610845,...,7.626083,30.698690,127.943185,15.992898,29.0,4.0,22.0,21.0,4.222222,1.444444


In [309]:
calDFFloat.loc[:,calDFFloat.isna().any()]

,AATS5dv,AATS6dv,AATS7dv,AATS8dv,AATS5d,AATS6d,AATS7d,AATS8d,AATS5s,AATS6s,...,MDEO-11,MDEO-12,MDEO-22,MDEN-11,MDEN-12,MDEN-13,MDEN-22,MDEN-23,MDEN-33,Vabc
CHEMBL222280,3.712801,4.694444,3.994553,3.164815,2.508772,2.431818,1.941176,1.600000,3.379305,4.863566,...,0.993957,1.043390,NaN,NaN,NaN,NaN,NaN,0.500000,NaN,205.926306
CHEMBL844,4.258398,4.053606,3.428571,2.458333,3.069767,2.921053,2.542857,2.500000,2.590547,2.587476,...,NaN,NaN,NaN,NaN,NaN,NaN,2.554030,NaN,NaN,198.391588
CHEMBL388590,4.763958,4.059259,2.524691,2.332672,3.298507,2.942857,2.430556,2.160714,3.512069,3.569665,...,0.166667,0.353553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276.665079
CHEMBL1201287,2.518913,1.995322,1.190719,0.379630,2.500000,2.389474,2.068966,1.644737,2.122669,2.166001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200000,NaN,264.237691
CHEMBL1199080,0.638218,0.338134,0.071429,0.000000,1.884058,1.685185,1.380952,1.166667,1.675188,1.794410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.245070
CHEMBL601719,3.927083,4.237513,4.006289,3.632008,2.843750,3.064220,2.933962,2.724771,2.871126,2.842829,...,NaN,NaN,NaN,NaN,0.559690,0.142857,0.493242,1.105209,NaN,368.818366
CHEMBL1293,1.097902,1.089552,1.276190,0.949640,1.895105,2.037313,2.071429,1.755396,1.801088,1.700249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,646.681902
CHEMBL841,2.624406,1.932000,1.451671,1.589099,2.581197,2.400000,2.252033,2.174528,2.463563,2.294028,...,0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.200000,456.434648
CHEMBL954,2.774123,2.835276,2.589744,1.934028,2.526316,2.608392,2.607692,2.304688,2.140919,2.264151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,397.116164
CHEMBL13960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.514560


In [310]:
calDF_test = calDFFloat[calDFFloat.columns[calDFFloat.isnull().mean() < .05]]

In [311]:
calDF_test

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
CHEMBL222280,13.003721,11.805899,0.0,0.0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,...,9.743730,63.794961,305.985134,10.928040,505.0,26.0,88.0,104.0,7.027778,3.777778
CHEMBL844,13.423098,11.209659,0.0,3.0,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,...,9.736193,63.007560,291.011957,10.778221,515.0,24.0,90.0,106.0,4.305556,3.750000
CHEMBL388590,17.125041,14.795147,0.0,0.0,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,...,10.136938,70.803080,421.915318,12.409274,988.0,38.0,118.0,143.0,8.000000,4.861111
CHEMBL1201287,14.389425,12.216867,0.0,1.0,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,...,9.572271,51.676805,318.073161,8.370346,718.0,25.0,92.0,103.0,6.305556,4.305556
CHEMBL1199080,9.579175,9.100648,0.0,1.0,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,...,9.256938,43.893749,242.053888,8.068463,248.0,16.0,62.0,68.0,5.784722,2.944444
CHEMBL601719,23.598392,18.449570,0.0,1.0,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,...,10.344545,79.445461,449.118544,8.636895,2633.0,49.0,160.0,190.0,9.722222,6.527778
CHEMBL1293,30.079292,22.385369,0.0,0.0,47.666770,2.391688,4.783375,47.666770,1.191669,4.572573,...,10.406473,76.538683,536.438202,5.587898,7984.0,57.0,194.0,216.0,16.013889,8.888889
CHEMBL841,26.472550,20.542158,0.0,1.0,44.176297,2.515749,5.031498,44.176297,1.299303,4.454465,...,10.600004,70.685296,476.223056,7.107807,3458.0,60.0,180.0,215.0,10.902778,7.486111
CHEMBL954,21.867940,17.016372,0.0,1.0,38.234665,2.413270,4.826539,38.234665,1.318437,4.272841,...,10.059765,63.939923,405.185942,7.108525,2488.0,43.0,142.0,163.0,8.527778,6.777778
CHEMBL13960,3.932653,4.244375,1.0,0.0,6.000000,2.000000,4.000000,6.000000,1.000000,2.610845,...,7.626083,30.698690,127.943185,15.992898,29.0,4.0,22.0,21.0,4.222222,1.444444


In [312]:
std_vals = calDF_test.std()

In [313]:
std_vals

ABC          7.634750
ABCGG        5.353569
nAcid        0.576766
nBase        0.814122
SpAbs_A     12.383404
              ...    
WPol        20.532119
Zagreb1     54.282335
Zagreb2     67.898093
mZagreb1     3.720608
mZagreb2     2.074155
Length: 1392, dtype: float64

In [314]:
## Remove std = 0 columns
calDF_ST = calDF_test[calDF_test.columns[calDF_test.std() != 0]]

In [315]:
calDF_ST

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
CHEMBL222280,13.003721,11.805899,0.0,0.0,21.163013,2.430127,4.790842,21.163013,1.244883,3.759503,...,9.743730,63.794961,305.985134,10.928040,505.0,26.0,88.0,104.0,7.027778,3.777778
CHEMBL844,13.423098,11.209659,0.0,3.0,22.818993,2.412409,4.818966,22.818993,1.342294,3.784126,...,9.736193,63.007560,291.011957,10.778221,515.0,24.0,90.0,106.0,4.305556,3.750000
CHEMBL388590,17.125041,14.795147,0.0,0.0,28.373722,2.499388,4.899958,28.373722,1.289715,4.030858,...,10.136938,70.803080,421.915318,12.409274,988.0,38.0,118.0,143.0,8.000000,4.861111
CHEMBL1201287,14.389425,12.216867,0.0,1.0,24.221083,2.349628,4.699256,24.221083,1.274794,3.848879,...,9.572271,51.676805,318.073161,8.370346,718.0,25.0,92.0,103.0,6.305556,4.305556
CHEMBL1199080,9.579175,9.100648,0.0,1.0,15.610374,2.304315,4.608630,15.610374,1.200798,3.457889,...,9.256938,43.893749,242.053888,8.068463,248.0,16.0,62.0,68.0,5.784722,2.944444
CHEMBL601719,23.598392,18.449570,0.0,1.0,39.093833,2.428055,4.836608,39.093833,1.303128,4.336994,...,10.344545,79.445461,449.118544,8.636895,2633.0,49.0,160.0,190.0,9.722222,6.527778
CHEMBL1293,30.079292,22.385369,0.0,0.0,47.666770,2.391688,4.783375,47.666770,1.191669,4.572573,...,10.406473,76.538683,536.438202,5.587898,7984.0,57.0,194.0,216.0,16.013889,8.888889
CHEMBL841,26.472550,20.542158,0.0,1.0,44.176297,2.515749,5.031498,44.176297,1.299303,4.454465,...,10.600004,70.685296,476.223056,7.107807,3458.0,60.0,180.0,215.0,10.902778,7.486111
CHEMBL954,21.867940,17.016372,0.0,1.0,38.234665,2.413270,4.826539,38.234665,1.318437,4.272841,...,10.059765,63.939923,405.185942,7.108525,2488.0,43.0,142.0,163.0,8.527778,6.777778
CHEMBL13960,3.932653,4.244375,1.0,0.0,6.000000,2.000000,4.000000,6.000000,1.000000,2.610845,...,7.626083,30.698690,127.943185,15.992898,29.0,4.0,22.0,21.0,4.222222,1.444444


In [224]:

## probably don't need this anymore
calDF_R = calDFFloat.replace({0:np.nan})
calDF_R.head()
calDF_RF = calDF_R[calDF_R.columns[calDF_R.isnull().mean() < .05]]
calDF_RF
calDF_RF_STD = calDF_RF[calDF_RF.columns[calDF_RF.std() != 0]]
calDF_RF_STD